In [444]:
using SparseArrays
using LinearAlgebra
using Random

In [445]:
Rx(theta) = exp(-1im*theta*[1 1;1 1]/2);
#Rx(theta) = [cos(theta/2) -1im*sin(theta/2) ; -1im*sin(theta/2)  cos(theta/2)];#

In [446]:
round.(-exp(-1im*pi*([1 1;1 1]/2)); digits = 3)

2×2 Matrix{ComplexF64}:
 -0.0+0.0im   1.0+0.0im
  1.0+0.0im  -0.0+0.0im

In [447]:
Ry(theta) = [cos(theta/2) -sin(theta/2) ; sin(theta/2) cos(theta/2)];

In [448]:
Pauli_Z = [1 0 ; 0 -1];

In [449]:
Hadamard(noise) = Ry(pi/2+noise)*Pauli_Z;

In [450]:
X = [0 1;1 0];

In [451]:
"""

Following function takes a 2x2 matrix (Gate) and qubit position (Qubit) and
returns the resultant matrix.

For example, the matrix for the gate U acting on the 3-rd qubit for N=5
qubit system is given by   I (x) I (x) U (x) I (x) I; where (x) is the
tensor product.

"""

function Matrix_Gate(Gate, Qubit) # Previously known as multi qubit gate.
    
    ## The case Qubit=1 is treated differently because we need to
    # initialize the matrix as U before starting the kronecker product.
    
    if Qubit == 1
        
        M = sparse(Gate)
        for i=2:L
            M = kron(M, sparse([1 0;0 1]))
        end
        
    else
        
        M = sparse([1 0;0 1])
        for i=2:L
            if i == Qubit
                M = kron(M, Gate)
            else
                M = kron(M, sparse([1 0;0 1]))
            end
        end
    end
    
    return M
end;

In [452]:
Identity(dimension) = 1* Matrix(I, dimension, dimension);
#Identity(3)

## Single controlled unitary gate

In [453]:
"""

The following function returns a controlled U gate matrix.

Input  : c (integer), t(integer), U (unitary operator).
Output : Matrix of the multicontrolled U gate with control qubit c and target qubit t.

"""

function CU(U,c,t)
    
    I2 = sparse([1 0;0 1])
    Z = sparse([1 0;0 -1])

    PI_0 = (I2+Z)/2
    PI_1 = (I2-Z)/2
     
    #function Rx(Noise)
        #A = cos((pi+Noise)/2)
        #B = -1im*sin((pi+Noise)/2)
        #return 1im*[A B;B A]
    #end
    
    Matrices = Dict("I" => I2,"PI_0" => PI_0,"U" => U, "PI_1" => PI_1)
    
    p0 = fill("I", L)
    p1 = fill("I", L)
    
    p0[c] = "PI_0"
    p1[c] = "PI_1"
    p1[t] = "U"

    
    PI_0_matrix = Matrices[p0[1]]
    for i = 2:L
        PI_0_matrix = kron(PI_0_matrix,Matrices[p0[i]])
    end        
        
    PI_1_matrix = Matrices[p1[1]]   
    for i = 2:L
        PI_1_matrix = kron(PI_1_matrix,Matrices[p1[i]])        
    end
           
    #return p0,p1
    return PI_0_matrix + PI_1_matrix     
end;               

## Multi controlled unitary gate

In [503]:
"""

The following returns a multicontrolled U gate matrix.

Input  : c (list), t(integer), U (unitary operator).
Output : Matrix of the multicontrolled U gate with control qubits c and target qubit t.

"""

function MCU(c,t,U)
    
    p0 = fill("I", L)
    p1 = fill("I", L)

    
    if typeof(c) == Int64
        p0[c] = "PI_1"
        p1[t] = "PI_1"
        
    else
        for i in c
            p0[i] = "PI_1"
            p1[i] = "PI_1"
        end
    end
    
    p0[t] = "I"
    p1[t] = "U"

    
    I = sparse([1 0;0 1])
    Z = sparse([1 0;0 -1])
    X = sparse([0 1;1 0])
    PI_0 = (I+Z)/2
    PI_1 = (I-Z)/2
     
    Matrices = Dict("I" => I,"PI_0" => PI_0,"U" => U, "PI_1" => PI_1)
    
    PI_0_matrix = Matrices[p0[1]]
    for i = 2:L
        PI_0_matrix = kron(PI_0_matrix,Matrices[p0[i]])
    end        
        
    PI_1_matrix = Matrices[p1[1]]   
    for i = 2:L
        PI_1_matrix = kron(PI_1_matrix,Matrices[p1[i]])        
    end
             
    # The identity in the following line needs to be replaced.
    return Identity(2^L) - PI_0_matrix + PI_1_matrix     
end;             

# Circuit for general L

In [504]:
L = 10
Number_Of_Gates = 2*L^2-6*L+5 + 2*(L+1)
Random.seed!(3000)
NOISE = 2*rand(Float64,Number_Of_Gates).-1;
#NOISE = zeros(Number_Of_Gates);

In [505]:
A = ones(2^L,2^L);
U_x = (2/2^L)*A-Identity(2^L);

In [506]:
function MCX_Reconstructed(DELTA)
    Noise_Counter = 1
    C_1 = [];
    C_2 = [];
    C_3 = [];
    C_4 = [];
    C_5 = [];
    C_6 = [];
    MCX = Identity(2^L);
    # C_1.
    for i = 1:L-2
        for j = 1:i
            push!(C_1,[j,L-i,L-i+j])
            
            epsilon = NOISE[Noise_Counter]
            MCX = CU(Rx((pi/2^j)+DELTA*epsilon), L-i, L-i+j)*MCX
            Noise_Counter += 1
            
        end
    end

    # C_2.
    for i = 2:L
        push!(C_2,[i-2,1,i])
        
        epsilon = NOISE[Noise_Counter]
        MCX = CU(Rx((pi/2^(i-2))+DELTA*epsilon), 1, i)*MCX
        Noise_Counter += 1
        
    end

    # C3 = - C1.
    for i = L-2:-1:1
        for j = i:-1:1
            push!(C_3,[j,L-i,L-i+j])
            
            epsilon = NOISE[Noise_Counter]
            MCX = CU(Rx((-pi/2^j)+DELTA*epsilon), L-i, L-i+j)*MCX
            Noise_Counter += 1
        end
    end

    # C_4.
    for i = 1:L-3
        for j = 1:i
            push!(C_4,[j,L-i-1,L-i+j-1])
            
            epsilon = NOISE[Noise_Counter]
            MCX = CU(Rx((pi/2^j)+DELTA*epsilon), L-i-1, L-i-1+j)*MCX
            Noise_Counter += 1
        end    
    end

    # C_5.
    for i = 2:L-1
        push!(C_5,[i-2,1,i])
        
        epsilon = NOISE[Noise_Counter]
        MCX = CU(Rx((-pi/2^(i-2))+DELTA*epsilon), 1, i)*MCX
        Noise_Counter += 1
        
    end

    # C6 = - C4.
    for i = L-3:-1:1
        for j = i:-1:1
            push!(C_6,[j,L-i-1,L-i-1+j])
            
            epsilon = NOISE[Noise_Counter]
            MCX = CU(Rx((-pi/2^j)+DELTA*epsilon), L-i-1, L-i-1+j)*MCX
            Noise_Counter += 1
            
        end    
    end
    
    return MCX
end    ;

In [507]:
# Total number of gates.
print("Total number of gates = ",2*L^2-6*L+5)

Total number of gates = 145

In [508]:
#MCX[2^L,2^L-1],MCX[2^L-1,2^L]
#length(C_1)+length(C_2)+length(C_3)+length(C_4)+length(C_5)+length(C_6)

In [509]:
XHL_Gates = []
for i = 1:L-1
    push!(XHL_Gates,["X",i])
end    
push!(XHL_Gates,["H",L])

XHR_Gates = [["H",L]]
for i = 1:L-1
    push!(XHR_Gates,["X",i])
end    

In [517]:
function U0_reconstructed(DELTA)
    
    Noise_Counter = 2*L^2-6*L+5;
    
    XHL_Matrix = Identity(2^L)
    for i in XHL_Gates
        
        if i[1] == "H"
            
            epsilon = NOISE[Noise_Counter]
            XHL_Matrix = XHL_Matrix*Matrix_Gate(Hadamard(DELTA*epsilon), i[2]) 
            Noise_Counter += 1 
            
        elseif i[1] == "X"
            
            epsilon = NOISE[Noise_Counter]
            XHL_Matrix = XHL_Matrix*Matrix_Gate(1im*Rx(pi+DELTA*epsilon),i[2])
            Noise_Counter += 1 
            
        end
    end
    
    XHR_Matrix = Identity(2^L)
    for j in XHR_Gates
        if j[1] == "H"
            
            epsilon = NOISE[Noise_Counter]
            XHR_Matrix = XHR_Matrix*Matrix_Gate(Hadamard(DELTA*epsilon), j[2]) 
            Noise_Counter += 1 
            
        elseif j[1] == "X"
            
            epsilon = NOISE[Noise_Counter]
            XHR_Matrix = XHR_Matrix*Matrix_Gate(1im*Rx(pi+DELTA*epsilon),j[2])
            Noise_Counter += 1 
        end
    end
    
    return XHL_Matrix*MCX_Reconstructed(DELTA)*XHR_Matrix
end;

In [512]:
#mcx = round.(MCX_Reconstructed(0.0);digits = 3);

In [513]:
#u0 = round.(U0_reconstructed(0.0); digits = 3);

In [515]:
Grover(DELTA) = U_x * U0_reconstructed(DELTA);

In [519]:
#Grover(0.1);